# Data-informed parameter synthesis for population Markov chains, notebook1

This notebook provides two first parts of analysis presented in the paper. 
1. [Creation of the models](#one)
2. [Parameter synthesis using prism](#two)

The second two parts are in following notebook.

Before running, please add [prism](prismmodelchecker.org) to PATH and check if it runs correctly.

There are 4 things to setup in the notebook:
1. `prism_path` -- path of prism - in order to import and call the prism properly please set up the paths (e.g. calling the prism outside )
2. `agents_quantities` -- set of agents number to be considered - set numbers for which you want to run analysis

## IMPORTS, PATHS, and SETTINGS

if a package fails to load due to it is not installed please uncomment and use following code with respective package name

In [1]:
import os

In [2]:
agents_quantities = [2,3,5,10]

In [1]:
#import sys
#!{sys.executable} -m pip install <package_name>

In [2]:
import struct
print("You are running "+ str(struct.calcsize("P") * 8)+"bit Python, please verify that installed z3 is compatible")

You are running 64bit Python, please verify that installed z3 is compatible


In [3]:
#imports evaluation
import time

In [4]:
#imports for creation of models
import math
import sys

In [5]:
#imports for running prism
import glob, os
import subprocess
import re

In [6]:
# import enhaced print
import socket

In [7]:
cwd = os.getcwd()

In [8]:
os.getcwd()

'D:\\Git\\hsb2019\\code'

Set path for prism

In [9]:
prism_path = ("c:/Program Files/prism-4.4/bin") #windows

#prism_path = ("/home/matej/prism-4.4-linux64/bin") #Freya

#prism_path = '/Users/tatjanapetrov/MYprism-4.4-osx64/bin' # mac@Uni
#os.environ['PATH'] = os.environ['PATH']+str(prism_path)

Checking if the prism in the PATH

In [10]:
if "prism" not in os.environ["PATH"]:
    print("prism was probably not in PATH, adding it there")
    if "wind" in platform.system().lower():
        os.environ["PATH"] = os.environ["PATH"]+";"+prism_path
    else:
        os.environ["PATH"] = os.environ["PATH"]+":"+prism_path

Set agents quantities to be examined

In [3]:
agents_quantities = [2,3,5,10]

<a name="one"></a>

## 1. CREATION OF THE MODELS

In this section are created two-param (p,q) and multiparametric models. Detailed description can be found in the paper in Section 2. For all models created in the notebook the bisimulation quotient has been applied. So to speeak, the states (1,0) and (0,1) are equal since the agent are the considered agents are identical. Hence we have merged those states.

The difference between synchronous, semisynchronous, and asynchronous models is in two parts of the model:
1. first attempt of the agent
2. second attempt 

synchronous models use synchronous semantics in the first attempt so does in the second attempt, these models are purely synchronous and differ from the other models in multiparametric case. 

asynchronous models use asynchronous semantics in the first attempt so does in the second attempt, these models are NOT purely synchronous as they assume synchronisation of the agents after first attempt! These models are largest and provide multi-affine labelling, hence are usable for STORM model checker. 

models called semisynchronous use synchronous semantics in the first attempt and asynchronous semantics in the second attempt. These models use synchronous semantics where the agents are independent and provide more precise asynchronous semantic when the order of the action is needed - Example: multiparametric case, two agents attempt second try while 3 agents has already succeeded - the first agent has probability q3 to succeeded, and the second q4 only if the first succeeded. We need more than 1 transition to describe this phenomena.

In [3]:
from src.create_models_props import nCr

### Creating models of synchronous two-param models

In [6]:
from src.create_models_props import create_synchronous_model

In [7]:
for N in agents_quantities:
    create_synchronous_model("synchronous_parallel_"+str(N),N)

synchronous_parallel_2
synchronous_parallel_3
synchronous_parallel_5
synchronous_parallel_10


### Creating models of semisynchronous two-param models

In [8]:
from src.create_models_props import create_semisynchronous_model

In [9]:
for N in agents_quantities:
    create_semisynchronous_model("semisynchronous_parallel_"+str(N),N)

semisynchronous_parallel_2
semisynchronous_parallel_3
semisynchronous_parallel_5
semisynchronous_parallel_10


### Creating models of asynchronous two-param models

In [10]:
from src.create_models_props import create_asynchronous_model

In [11]:
for N in agents_quantities:
    create_asynchronous_model("asynchronous_"+str(N),N)

asynchronous_2
asynchronous_3
asynchronous_5
asynchronous_10


### Creating models of synchronous multiparam models

In [12]:
from src.create_models_props import create_multiparam_synchronous_model

In [13]:
for N in agents_quantities:
    create_multiparam_synchronous_model("multiparam_synchronous_parallel_"+str(N),N)

multiparam_synchronous_parallel_2
multiparam_synchronous_parallel_3
multiparam_synchronous_parallel_5
multiparam_synchronous_parallel_10


### Creating models of semisynchronous multi-param models

In [14]:
from src.create_models_props import create_multiparam_semisynchronous_model

In [15]:
for N in agents_quantities:
    create_multiparam_semisynchronous_model("multiparam_semisynchronous_parallel_"+str(N),N)

multiparam_semisynchronous_parallel_2
multiparam_semisynchronous_parallel_3
multiparam_semisynchronous_parallel_5
multiparam_semisynchronous_parallel_10


### Creating models of asynchronous multi-param models

In [16]:
from src.create_models_props import create_multiparam_asynchronous_model

In [17]:
for N in agents_quantities:
    create_multiparam_asynchronous_model("multiparam_asynchronous_"+str(N),N)

multiparam_asynchronous_2
multiparam_asynchronous_3
multiparam_asynchronous_5
multiparam_asynchronous_10


### Creating properties files

In [18]:
from src.create_models_props import create_properties

In [19]:
for N in agents_quantities:
    create_properties(N)

prop_2
prop_3
prop_5
prop_10


<a name="two"></a>

## 2. PARAMETER SYNTHESIS

In this section parameter synthesis is run and the results of it with the data are loaded.

The properties are reachability properties of respective BSCC of the model.

The result are rational function, in this case study polynomes over parameter p,q or p,q1,q2... in multipram case.

In [21]:
os.mkdir("prism_results")

In [22]:
os.getcwd()

'C:\\Users\\xhajnal\\Documents\\GitHub\\mpm\\ipython'

### Using PRISM

In [24]:
from src.mc_prism import call_prism

Call prism for all files in dir and Saves result to folder `prism_results`. If error with rounding occurs `-noprobchecks` attribute is added. If Java runs out of nemory prism is called with each property separately. 

Example: `prism semisynchronous_parallel_2.txt prop_2.pctl -param p=0:1,q=0:1`

In [3]:
from src.mc_prism import call_prism_files

### Parameter synthesis of synchronous two_param models

In [4]:
call_prism_files("syn*_",False,agents_quantities)

synchronous_parallel_2.pm seq=False


NameError: name 'prism_path' is not defined

### Parameter synthesis of semisynchronous two_param models

In [119]:
call_prism_files("semi*_",False)

semisynchronous_parallel_2.pm seq=False
  It took Thunder 2.300849199295044 seconds to run
semisynchronous_parallel_3.pm seq=False
  It took Thunder 2.2050530910491943 seconds to run
semisynchronous_parallel_5.pm seq=False
  It took Thunder 2.1482558250427246 seconds to run
semisynchronous_parallel_10.pm seq=False
  It took Thunder 2.1563894748687744 seconds to run


### Parameter synthesis of asynchronous two_param models

In [120]:
call_prism_files("asyn*_",False)

asynchronous_2.pm seq=False
  It took Thunder 2.3576972484588623 seconds to run
asynchronous_3.pm seq=False
  It took Thunder 2.172189474105835 seconds to run
asynchronous_5.pm seq=False
  It took Thunder 2.423518657684326 seconds to run
asynchronous_10.pm seq=False
  It took Thunder 2.5880792140960693 seconds to run


### Parameter synthesis of synchronous multiparam models

In [136]:
call_prism_files("multiparam_syn*_",True)

multiparam_synchronous_parallel_2.pm seq=False
  It took Thunder 2.142266035079956 seconds to run
multiparam_synchronous_parallel_3.pm seq=False
  It took Thunder 2.1333208084106445 seconds to run
multiparam_synchronous_parallel_5.pm seq=False
  It took Thunder 2.134281873703003 seconds to run
multiparam_synchronous_parallel_10.pm seq=False
  It took Thunder 2.157231092453003 seconds to run


### Parameter synthesis of semisynchronous multiparam models

In [122]:
call_prism_files("multiparam_se*_",True)

multiparam_semisynchronous_parallel_2.pm seq=False
  It took Thunder 2.14426326751709 seconds to run
multiparam_semisynchronous_parallel_3.pm seq=False
  It took Thunder 2.131301164627075 seconds to run
multiparam_semisynchronous_parallel_5.pm seq=False
  It took Thunder 2.1512391567230225 seconds to run
multiparam_semisynchronous_parallel_10.pm seq=False
  It took Thunder 2.322788953781128 seconds to run


### Parameter synthesis of asynchronous multiparam models

In [123]:
call_prism_files("multiparam_a*_",True)

multiparam_asynchronous_2.pm seq=False
  It took Thunder 2.178176164627075 seconds to run
multiparam_asynchronous_3.pm seq=False
  It took Thunder 2.2250471115112305 seconds to run
multiparam_asynchronous_5.pm seq=False
  It took Thunder 2.1782054901123047 seconds to run
multiparam_asynchronous_10.pm seq=False
  It took Thunder 6.811779737472534 seconds to run


All models and properties created, parameter synthesis for all models is done, please follow notebook **Analysis**.